In [1]:
pip install SQLAlchemy==1.4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.17-cp310-cp310-linux_x86_64.whl size=1519915 sha256=b13652ade64acce78ff018bbffeb9494a9466f2ed2fc4b9d9de02f8fc6a6e6b6
  Stored in directory: /root/.cache/pip/wheels/eb/d0/1e/ab0fcbac6f74a69b788a37fccd365fc67535b5ab79f97bd860
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.17 which is incompatible.


In [2]:
pip install pymssql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 15.7 MB/s eta 0:00:00


In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.8 MB/s eta 0:00:00


In [4]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import statsmodels.formula.api as smf
from pandas_datareader import DataReader as pdr

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as rf
from catboost import CatBoostRegressor as sbr

In [5]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912"
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

## Strategy 2

In [6]:
df = pd.read_sql(
    """
    select date, ticker, mom1m, mom12m, mom36m, siccd, ret, sfe, rsup, ear, chmom, idiovol
    from data
    order by date, ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index(["date", "ticker"])
features = ["mom36m", "sfe", "rsup", "ear", "chmom", "idiovol"]

In [7]:
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)
df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d:
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )
)

In [8]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes = pd.DataFrame(codes).reset_index()
codes.columns = ["siccd", "industry"]

df = df.reset_index().merge(codes, on="siccd")
df = df.set_index(["date", "ticker"])

features.append("industry")

Pipeline 1: Also favourite pipeline

quantile transform + column transform + polynomial transform + model

In [9]:
pipe1 = sbr(cat_features=["industry"], random_state=42)


Looping test

## Strategy 3

In [10]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None
train_score = []
test_score = []

for train_date, end_date in zip(dates[:-1], dates[1:]):

  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]

  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  ytest = test["ret"]

  pipe1.fit(Xtrain, ytrain)
  print('Train set score: ' + str(pipe1.score(Xtrain, ytrain)))


  pred = pipe1.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
  print('Test set score: ' + str(pipe1.score(Xtest,ytest)))

df["predict"] = predictions

Learning rate set to 0.091534
0:	learn: 1.0061818	total: 237ms	remaining: 3m 56s
1:	learn: 1.0057155	total: 346ms	remaining: 2m 52s
2:	learn: 1.0052746	total: 472ms	remaining: 2m 36s
3:	learn: 1.0049127	total: 604ms	remaining: 2m 30s
4:	learn: 1.0046124	total: 703ms	remaining: 2m 19s
5:	learn: 1.0043628	total: 794ms	remaining: 2m 11s
6:	learn: 1.0040993	total: 898ms	remaining: 2m 7s
7:	learn: 1.0038635	total: 1.01s	remaining: 2m 4s
8:	learn: 1.0036514	total: 1.13s	remaining: 2m 4s
9:	learn: 1.0034910	total: 1.23s	remaining: 2m 2s
10:	learn: 1.0033203	total: 1.34s	remaining: 2m
11:	learn: 1.0031688	total: 1.44s	remaining: 1m 58s
12:	learn: 1.0030510	total: 1.54s	remaining: 1m 56s
13:	learn: 1.0029175	total: 1.63s	remaining: 1m 55s
14:	learn: 1.0028070	total: 1.74s	remaining: 1m 53s
15:	learn: 1.0027054	total: 1.85s	remaining: 1m 53s
16:	learn: 1.0026155	total: 1.98s	remaining: 1m 54s
17:	learn: 1.0024787	total: 2.06s	remaining: 1m 52s
18:	learn: 1.0023746	total: 2.12s	remaining: 1m 49s


In [11]:
df.to_csv('strat3.csv')

In [12]:
numlong = 150
numshort = 100

df["rank_from_top"] = df.groupby("date").predict.rank(
  method="first",
  ascending=False
)
df["long"] = df.rank_from_top <= numlong

df["rank_from_bottom"] = df.groupby("date").predict.rank(
  method="first"
)
df["short"] = df.rank_from_bottom <= numshort

In [13]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
df = df[df.index.get_level_values("date") >= dates[0]]

long_ret = df[df.long].groupby("date").actual.mean()
short_ret = df[df.short].groupby("date").actual.mean()

In [14]:
ret = 1.5*long_ret - 0.5*short_ret
ret

date
2005-01   -0.009409
2005-02    0.039488
2005-03    0.008699
2005-04   -0.048090
2005-05    0.096987
             ...   
2021-11   -0.019711
2021-12    0.104777
2022-01   -0.033037
2022-02    0.017081
2022-03   -0.010336
Name: actual, Length: 207, dtype: float64

In [15]:
df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
df_new.index = df_new.index.astype(str)
df_new["ret"] = ret
df_new["ret_rf"] = df_new.ret - df_new.RF
df_new = df_new.dropna()
df_new.index = df_new.index.astype(str)
df_new.index.name = "date"
df_new = df_new.reset_index()
df_new = df_new.rename(columns={"Mkt-RF": "mkt_rf", "RF": "rf"})

<ipython-input-15-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
<ipython-input-15-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100


In [16]:
#Computing alpha
result = smf.ols("ret_rf~mkt_rf+SMB+HML+CMA+RMW", df_new).fit()
betas = result.params[1:]
mkt = df_new.rf + betas[0]*df_new.mkt_rf
smb = betas[1]*df_new.SMB
hml = betas[2]*df_new.HML
cma = betas[3]*df_new.CMA
rmw = betas[4]*df_new.RMW
active = df_new.ret - mkt - smb - hml - cma - rmw


In [17]:
print("Active is found to be", active)

Active is found to be 0      0.014755
1      0.014479
2      0.027025
3     -0.006255
4      0.047114
         ...   
202   -0.001404
203    0.062472
204    0.012606
205    0.020189
206   -0.026049
Length: 207, dtype: float64


In [18]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 ret_rf   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     97.61
Date:                Thu, 02 May 2024   Prob (F-statistic):           7.81e-52
Time:                        22:24:41   Log-Likelihood:                 407.48
No. Observations:                 207   AIC:                            -803.0
Df Residuals:                     201   BIC:                            -783.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0066      0.003      2.629      0.0

In [19]:
df

,,mom1m,mom12m,mom36m,siccd,ret,sfe,rsup,ear,chmom,idiovol,actual,industry,predict,rank_from_top,long,rank_from_bottom,short
date,ticker,,,,,,,,,,,,,,,,,
2005-01,AA,-0.075338,-0.089728,-0.239517,3334,-0.328580,1.686779,0.041611,0.250438,0.162705,-0.755433,-0.060789,Manufacturing,0.008826,1640.0,False,1261.0,False
2005-02,AA,-0.060789,-0.064237,-0.560558,3334,1.006589,1.754757,0.052713,0.253814,-0.657817,-0.741304,0.093528,Manufacturing,0.056839,899.0,False,1978.0,False
2005-03,AA,0.093528,-0.201487,-0.540889,3334,-0.463957,1.734338,0.400524,-0.272973,-0.215342,-0.736432,-0.053861,Manufacturing,0.036430,1266.0,False,1599.0,False
2005-04,AA,-0.053861,-0.056828,-0.576793,3334,0.052657,1.673123,0.415254,-0.259957,-0.668685,-0.704783,-0.045081,Manufacturing,0.091157,519.0,False,2350.0,False
2005-05,AA,-0.045081,0.006712,-0.597934,3334,-1.335579,1.662696,0.421735,-0.262490,-0.585034,-0.655330,-0.060992,Manufacturing,0.085100,579.0,False,2120.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11,KSPN,-0.129874,0.481778,-0.875935,5735,-0.316798,-1.154158,1.813806,-0.284599,-1.322762,1.962924,-0.062392,Shops,-0.133764,1864.0,False,192.0,False
2021-12,KSPN,-0.062392,0.135848,-0.961365,5735,-1.866321,-1.149192,1.443499,-0.436823,-1.613696,1.951665,-0.232353,Shops,-0.008860,1427.0,False,619.0,False
2022-01,KSPN,-0.232353,-0.070403,-0.224912,5735,-1.916238,-1.185322,-2.378046,-0.949205,-1.652350,1.966406,-0.281609,Shops,0.179395,7.0,True,2017.0,False
